## Importing pydicom, neccessary libraries and loading data
* Dicom (Digital Imaging in Medicine) is the bread and butter of medical image datasets, storage and transfer.
* Reading dicom patients liver masks files 

In [ ]:
import os 
import pandas as pd 
import cv2
import numpy as np
import scipy.ndimage
import matplotlib.pyplot as plt
import pydicom

In [ ]:
liver_segm = []
for i in range(15):
    s1 = "3Dircadb1/3Dircadb1."
    s2 = "/MASKS_DICOM/liver"
    liver_segm.append(s1+str(i+1)+s2)


In [ ]:
liver_slices =[]
for liver in liver_segm:
#     labels_df.get_value(patient, 'cancer')
    path = liver
    # a couple great 1-liners from: https://www.kaggle.com/gzuidhof/data-science-bowl-2017/full-preprocessing-tutorial
    slices = [pydicom.read_file(path+"/" + s) for s in os.listdir(path)]
    slices.sort(key = lambda x: int(x.ImagePositionPatient[2]))
    liver_slices.append(slices)


## Hounsfield Units 

* The unit of measurement in CT scans is the Hounsfield Unit (HU), which is a measure of radiodensity. CT scanners are carefully calibrated to accurately measure this. From Wikipedia:

* By default however, the returned values are not in this unit. Let's fix this.

* Some scanners have cylindrical scanning bounds, but the output image is square. The pixels that fall outside of these bounds get the fixed value -2000. The first step is setting these values to 0, which currently corresponds to air. Next, let's go back to HU units, by multiplying with the rescale slope and adding the intercept (which are conveniently stored in the metadata of the scans!).

In [ ]:
#returns the hounse fied unit of slices 
def get_pixels_hu(slices):
    image = np.stack([s.pixel_array for s in slices])
    # Convert to int16 (from sometimes int16), 
    # should be possible as values should always be low enough (<32k)
    image = image.astype(np.int16)

    # Set outside-of-scan pixels to 0
    # The intercept is usually -1024, so air is approximately 0
    image[image == -2000] = 0
    
    # Convert to Hounsfield units (HU)
    for slice_number in range(len(slices)):
        
        intercept = slices[slice_number].RescaleIntercept
        slope = slices[slice_number].RescaleSlope
        
        if slope != 1:
            image[slice_number] = slope * image[slice_number].astype(np.float64)
            image[slice_number] = image[slice_number].astype(np.int16)
            
        image[slice_number] += np.int16(intercept)
    
    return np.array(image, dtype=np.int16)

liver_pixels = []
for liver in liver_slices:
    pixels = get_pixels_hu(liver)
    liver_pixels.append(pixels)

## Normalization
* A commonly used set of thresholds to normalize is -200 and 500.
* Our values currently range from -1024 to around 2000. Anything above 500 is not interesting to us these are simply bones with different radiodensity, and below -200 is just fats, lungs and air radiodensties. 
*  Here's some code you can use

In [ ]:
# normalize the liver pixels 
def normalize2(image):
    MAX_BOUND = np.max(image)
    MIN_BOUND = np.min(image)
    image = (image - MIN_BOUND) / (MAX_BOUND - MIN_BOUND)
    image[image>1] = 1.
    image[image<0] = 0.
    return image


normalized_liver = []
for pat in liver_pixels:
    normalized_liver.append( normalize2(pat) )
    

## Resampling

* A scan may have a pixel spacing of [2.5, 0.5, 0.5], which means that the distance between slices is 2.5 millimeters. For a different scan this may be [1.5, 0.725, 0.725], this can be problematic for automatic analysis (e.g. using ConvNets)!

* A common method of dealing with this is resampling the full dataset to a certain isotropic resolution. If we choose to resample everything to 1mm1mm1mm pixels we can use 3D convnets without worrying about learning zoom/slice thickness invariance.

* Whilst this may seem like a very simple step, it has quite some edge cases due to rounding. Also, it takes quite a while.


In [ ]:
#resample the liver image to 1mm in all diminsions 
def resample(image, scan, new_spacing=[1,1,1]):
    # Determine current pixel spacing
    spacing = np.array([scan[0].SliceThickness, scan[0].PixelSpacing[0], scan[0].PixelSpacing[1]], dtype=np.float32)

    resize_factor = spacing / new_spacing
    new_real_shape = image.shape * resize_factor
    new_shape = np.round(new_real_shape)
    real_resize_factor = new_shape / image.shape
    new_spacing = spacing / real_resize_factor 
    image = scipy.ndimage.interpolation.zoom(image, real_resize_factor, mode='nearest')
    image = image.astype(np.uint8)
    
    
    return image


resampled_liver_pixels = []
for i in range(15):
    pix_resampled = resample(normalized_liver[i], liver_slices[i], [1,1,1])
    resampled_liver_pixels.append(pix_resampled)

## Saving preproccessed patients liver masks offline

* preproccessing is a time and memory consuming proccess so we prefare to save our clean and ready data offline


In [ ]:
np.save('offline-patients.npy',resampled_liver_pixels)